In [1]:
import datetime
import geopandas
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2023-05-04 20:47:49.081542: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# TODO try only with best performing numeric columns for input into model first
# TODO select features for deep learning model after ranking features with random forest (feature importance)
# TODO lstm network
# TODO SMOTE algorithm, for upsampling havested fields; we only have ~300 :(

In [3]:
#local scripts
from scripts import veg_indices, utilities, plots
#from scripts.utilities import *

In [4]:
# cpied from learning_about-data.ipynb
BANDS_DICT = {   'B2': 'Blue',
            'B3': 'Green',
            'B4': 'Red',
            'B5': 'Red_Edge_1',
            'B6': 'Red_Edge_2',
            'B7': 'Red_Edge_3',
            'B8': 'NIR',
            'B8A': 'Red_Edge_4',
            'B11': 'SWIR_1',
            'B12': 'SWIR_2'}

BANDS = list(BANDS_DICT.keys())

In [5]:
# cpied from learning_about-data.ipynb
DF = geopandas.read_file('../data/merged_images.geojson')
DF.rename(columns = {'is_within_period':'har_evnt'}, inplace = True)
NUM_SAMPLES = len(np.unique(DF.image_idx)) - 1

In [204]:
# cpied from learning_about-data.ipynb
df = DF.copy()
df = df[(df.NDVI) != 0] # drop invalid points
VEG_INDICES_NAMES = veg_indices.add_veg_indices(df) + ['NDVI'] 
VEG_DIFF_NAMES = veg_indices.add_veg_diff(df, VEG_INDICES_NAMES)
NUMERIC_COLS = BANDS + VEG_INDICES_NAMES + VEG_DIFF_NAMES
df = df.dropna(subset=['sample_idx']) # removes the first image, because sample_idx is NaN there


#""" 
# only look at samples 6, 7, 8, 9
new_df = None
for i in range(6, 9 + 1):

    curr_df = df[df["sample_idx"] == f"s{i}"]
    if(type(new_df) == type(None)):
        new_df = curr_df
    else:
        new_df = pd.concat([new_df, curr_df])
df = new_df
#"""

print(df.columns, df.shape)


# For each 3-week image, standarize each column
df = utilities.get_rm_outlier_standarize(df, NUMERIC_COLS, rm_outliers=False)

Added:  ['DVI', 'RVI', 'ARVI', 'PSSRa', 'NDI45', 'GNDVI', 'MCARI', 'IRECI', 'CIr', 'MTCI', 'NDVIre', 'NIRv', 'EVI', 'NDTI', 'NDMI', 'MSI', 'GCI', 'NBRI', 'BSI', 'NDWI', 'NDSI']
Added:  ['sample_idx', 'DVI_diff', 'RVI_diff', 'ARVI_diff', 'PSSRa_diff', 'NDI45_diff', 'GNDVI_diff', 'MCARI_diff', 'IRECI_diff', 'CIr_diff', 'MTCI_diff', 'NDVIre_diff', 'NIRv_diff', 'EVI_diff', 'NDTI_diff', 'NDMI_diff', 'MSI_diff', 'GCI_diff', 'NBRI_diff', 'BSI_diff', 'NDWI_diff', 'NDSI_diff', 'NDVI_diff']
Index(['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'NDVI',
       'finHarvDat', 'lat', 'lon', 'point_idx', 'start_date', 'end_date',
       'har_evnt', 'image_idx', 'geometry', 'DVI', 'RVI', 'ARVI', 'PSSRa',
       'NDI45', 'GNDVI', 'MCARI', 'IRECI', 'CIr', 'MTCI', 'NDVIre', 'NIRv',
       'EVI', 'NDTI', 'NDMI', 'MSI', 'GCI', 'NBRI', 'BSI', 'NDWI', 'NDSI',
       'pt_idx', 'img_idx', 'DVI_diff', 'RVI_diff', 'ARVI_diff', 'PSSRa_diff',
       'NDI45_diff', 'GNDVI_diff', 'MCARI_diff', 'IRECI_d

In [205]:
def get_one_hot(df:pd.DataFrame, colName: str, retArrays=False):
    lookupTable, indexed_dataSet = np.unique(df[colName], return_inverse=True)
    tpls = tf.keras.utils.to_categorical(indexed_dataSet)

    arrays = tpls.T[1:] # remove one column (not needed)
    arrays = arrays.astype(np.float32) # tensorflow does not support bools, so float 32 is the best we can do :/
    if(retArrays):
        return arrays

    return arrays.T # return to n by m (n tuples of size m)




In [206]:
df

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,...,NDTI_diff,NDMI_diff,MSI_diff,GCI_diff,NBRI_diff,BSI_diff,NDWI_diff,NDSI_diff,NDVI_diff,sample_idx
3906,-0.870086,-0.894463,-0.362898,-0.562899,-0.712486,-0.803480,-0.029281,0.751010,0.946631,0.729749,...,-1.478007,1.603634,-1.360793,2.700617,-2.968218,-1.523347,0.521851,1.005915,1.573112,s6
3907,-0.312600,-0.369832,-0.086144,0.280409,-0.099855,0.208097,0.629703,0.362093,0.061773,0.152029,...,0.003610,0.370216,-0.424915,-0.590995,0.906889,-0.333211,1.112289,0.629439,-0.610115,s6
3908,0.709577,0.053909,-0.152589,-0.109988,-0.493141,-0.196534,1.214141,1.359916,1.234850,1.216424,...,-1.587707,-0.081406,-0.394088,0.704432,-0.719594,-0.579386,-0.135283,0.200484,1.265219,s6
3909,-0.561720,-0.722576,-0.312302,-0.473593,-0.729585,-0.671262,0.668467,1.463120,2.055841,1.519354,...,-2.537627,1.919438,-1.531427,3.167865,-3.512676,-1.735957,0.806501,1.165113,1.692973,s6
3910,-0.299037,-0.194955,-0.324494,-0.356218,-0.293845,-0.373203,-0.754462,-0.892654,-0.904964,-0.916228,...,0.557096,-0.683240,0.500220,-0.312166,0.231992,0.551830,-0.896480,-0.580907,0.203664,s6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6133,-1.223664,-1.653289,-0.874174,-0.848826,-1.376933,-0.955374,1.352084,1.767256,1.749997,1.694362,...,-1.565219,1.862842,-1.964314,2.941633,-3.306606,-2.374764,0.411842,2.136983,2.313166,s9
6134,0.585052,0.995327,-0.201453,-0.167603,0.086507,-0.184328,-0.739681,-0.749316,-0.777399,-0.679339,...,0.142589,-0.838315,1.168225,-0.907196,0.397156,1.137787,-0.727334,-1.122512,-0.399597,s9
6135,-0.186510,0.046045,-0.259642,-0.345125,0.100141,-0.173940,-0.687242,-0.689586,-0.657617,-0.606040,...,0.554367,-0.367829,0.398082,-0.382788,0.256747,0.560347,-0.615972,-0.461928,-0.459774,s9
6136,0.421999,1.102268,-0.817404,-1.045111,-0.845186,-1.091576,-1.444705,-1.206955,-1.128188,-1.259767,...,-0.706420,-1.028375,1.619866,-0.555706,-0.749974,1.448490,-2.587140,-1.076003,0.980113,s9


In [207]:


def convert_to_1hot(categorical_col_name:str):
    names = []
    arrays = get_one_hot(df, categorical_col_name, retArrays=True)
    for i in range(len(arrays)):
        arr = arrays[i]
        name = f'{categorical_col_name}_1hot_{i}'
        df[name] = arr
        names.append(name)
    return names

one_hot_names_X = convert_to_1hot("sample_idx")
one_hot_names_Y = convert_to_1hot("har_evnt")

print(df.columns)
print(one_hot_names_X)
print(one_hot_names_Y)


Index(['B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'NDVI',
       'finHarvDat', 'lat', 'lon', 'point_idx', 'start_date', 'end_date',
       'har_evnt', 'image_idx', 'geometry', 'DVI', 'RVI', 'ARVI', 'PSSRa',
       'NDI45', 'GNDVI', 'MCARI', 'IRECI', 'CIr', 'MTCI', 'NDVIre', 'NIRv',
       'EVI', 'NDTI', 'NDMI', 'MSI', 'GCI', 'NBRI', 'BSI', 'NDWI', 'NDSI',
       'pt_idx', 'img_idx', 'DVI_diff', 'RVI_diff', 'ARVI_diff', 'PSSRa_diff',
       'NDI45_diff', 'GNDVI_diff', 'MCARI_diff', 'IRECI_diff', 'CIr_diff',
       'MTCI_diff', 'NDVIre_diff', 'NIRv_diff', 'EVI_diff', 'NDTI_diff',
       'NDMI_diff', 'MSI_diff', 'GCI_diff', 'NBRI_diff', 'BSI_diff',
       'NDWI_diff', 'NDSI_diff', 'NDVI_diff', 'sample_idx',
       'sample_idx_1hot_0', 'sample_idx_1hot_1', 'sample_idx_1hot_2',
       'har_evnt_1hot_0'],
      dtype='object')
['sample_idx_1hot_0', 'sample_idx_1hot_1', 'sample_idx_1hot_2']
['har_evnt_1hot_0']


In [183]:
#  reshape the data into a 3D tensor with shape (num_samples, sequence_length, num_features) to feed it into a 1D CNN in TensorFlow for training.
# sequence_length: each 3-week sample
# num_samples: num of points, aka farms

points_dfs_X = []
points_dfs_Y = []

for p in sorted(df['point_idx'].unique(), key= lambda x: int(x[1:])):
    curr_df = df[df["point_idx"] == p]

    if(curr_df.shape[0] == 4): #  TODO fix this, even method1 is not done correctly :/ 
        """
        TODO
        use one of the follwoing(currently using method 1):
            1-Remove the rows with missing values.
            2-Fill in missing values with a constant
            3-Interpolate missing values
            4-Use imputation methods
        """
        curr_df_X = (curr_df[one_hot_names_X + BANDS]).to_numpy()
        points_dfs_X.append(curr_df_X) 

        curr_df_Y = (curr_df[one_hot_names_Y]).to_numpy()
        points_dfs_Y.append(curr_df_Y) 




In [184]:
X = np.array(points_dfs_X)
Y = np.array(points_dfs_Y)

In [185]:
X.shape

(430, 4, 13)

In [186]:
Y.shape

(430, 4, 1)

In [187]:
Y = Y.reshape((430, 4))
Y

array([[0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.]], dtype=float32)

In [188]:
X.shape[1], X.shape[2]

(4, 13)

In [218]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization

model = tf.keras.Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(4, 13)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    BatchNormalization(),
    Dense(units=64, activation='relu'),
    Dense(units=256, activation='relu'),
    Dense(units=256, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=4)
])

#model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [219]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


In [220]:
model.fit(X, Y, epochs=200, validation_split=0.1)


Epoch 1/200
13/13 [==============================] - 2s 28ms/step - loss: 0.1411 - accuracy: 0.3566 - val_loss: 0.1068 - val_accuracy: 0.3023
Epoch 2/200
13/13 [==============================] - 0s 8ms/step - loss: 0.1111 - accuracy: 0.4289 - val_loss: 0.1056 - val_accuracy: 0.3488
Epoch 3/200
13/13 [==============================] - 0s 10ms/step - loss: 0.1055 - accuracy: 0.4315 - val_loss: 0.1053 - val_accuracy: 0.3256
Epoch 4/200
13/13 [==============================] - 0s 8ms/step - loss: 0.0978 - accuracy: 0.4496 - val_loss: 0.1055 - val_accuracy: 0.3256
Epoch 5/200
13/13 [==============================] - 0s 9ms/step - loss: 0.0928 - accuracy: 0.4574 - val_loss: 0.0984 - val_accuracy: 0.3256
Epoch 6/200
13/13 [==============================] - 0s 10ms/step - loss: 0.0890 - accuracy: 0.4987 - val_loss: 0.0993 - val_accuracy: 0.3721
Epoch 7/200
13/13 [==============================] - 0s 9ms/step - loss: 0.0821 - accuracy: 0.4935 - val_loss: 0.0943 - val_accuracy: 0.3256
Epoch 8/20

In [216]:
a = tf.constant(X)
tf.math.l2_normalize(a).numpy()

array([[[ 0.        ,  0.        ,  0.        , ...,  0.00552828,
         -0.00659141, -0.00677609],
        [ 0.00757559,  0.        ,  0.        , ..., -0.00238965,
         -0.01646883, -0.01451381],
        [ 0.        ,  0.00757559,  0.        , ..., -0.00472415,
         -0.00212759, -0.00230309],
        [ 0.        ,  0.        ,  0.00757559, ...,  0.00232412,
          0.00348978,  0.00299009]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.00921513,
          0.00537546,  0.00040839],
        [ 0.00757559,  0.        ,  0.        , ...,  0.0002729 ,
          0.00560671,  0.00508392],
        [ 0.        ,  0.00757559,  0.        , ..., -0.00203002,
          0.00129858,  0.00159386],
        [ 0.        ,  0.        ,  0.00757559, ...,  0.01397219,
         -0.00468563, -0.00910651]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.01151   ,
         -0.00425536, -0.00547394],
        [ 0.00757559,  0.        ,  0.        , ...,  0.00533579,
         -0.

In [75]:
from sklearn.model_selection import train_test_split

trainX, testX, trainy, testy =  train_test_split(X, Y, test_size = 0.4,random_state=42) 

evaluate_model(trainX, trainy, testX, testy)

ValueError: in user code:

    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/tahaalnufaili/Library/Caches/pypoetry/virtualenvs/harvest-event-detec-Fyp2lGEN-py3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Users/tahaalnufaili/

In [ ]:
testX.shape, trainy.shape

((1249, 1), (834, 57))